<a href="https://colab.research.google.com/github/YasminBougammoura/nlp/blob/main/QuantizedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
%reset -f

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [81]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-3
eval_iters = 200
n_embd = 32
eps = 1e-3
n_head = 4
n_layer = 4
dropout = 0.2

In [3]:
with open('input.txt','r', encoding = 'utf-8') as f:
    text = f.read()

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [5]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    qmodel.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = qmodel(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    qmodel.train()
    return out

In [7]:
class BitLinear(nn.Module):

  def __init__(self, in_features, out_features, bits):
    super().__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.bits = bits
    self.weight = nn.Parameter(torch.zeros(out_features, in_features))
    nn.init.kaiming_uniform_(self.weight, a = math.sqrt(5))

  def quantization(self, input_data):
    n,m = self.weight.shape
    mean_adj = torch.sum(self.weight)/(n*m)
    adj_par = 2**(self.bits-1)/torch.max(torch.abs(input_data))
    Wq = torch.sign(self.weight - mean_adj)
    Xq = torch.clip(input_data*adj_par, -2**(self.bits-1)+eps, 2**(self.bits-1)-eps)
    return Xq,Wq

  def forward(self,x):
    ln = nn.LayerNorm(x.shape[2])
    X_q,W_q = self.quantization(ln(x))
    gamma = torch.max(torch.abs(x))
    beta = torch.sum(torch.abs(self.weight))/self.weight.shape[0]*self.weight.shape[1]
    deq_par = beta * gamma / 2**(self.bits-1)

    y = W_q @ X_q.transpose(-2,-1) #* deq_par
    y = y.transpose(-2,-1)
    return y


In [47]:
class BitLinearReLU(nn.Module):

  def __init__(self, in_features, out_features, bits):
    super().__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.bits = bits
    self.weight = nn.Parameter(torch.zeros(out_features, in_features))
    nn.init.kaiming_uniform_(self.weight, a = math.sqrt(5))

  def quantization(self, input_data):
    n,m = self.weight.shape
    mean_adj = torch.sum(self.weight)/(n*m)
    adj_par = 2**(self.bits-1)/torch.max(torch.abs(input_data))
    Wq = torch.sign(self.weight - mean_adj)
    Xq = torch.clip((input_data-torch.min(input_data))*adj_par, eps, 2**(self.bits-1)-eps)
    return Xq,Wq

  def forward(self,x):
    ln = nn.LayerNorm(x.shape[2])
    X_q,W_q = self.quantization(ln(x))
    gamma = torch.max(torch.abs(x))
    beta = torch.sum(torch.abs(self.weight))/self.weight.shape[0]*self.weight.shape[1]
    deq_par = beta * gamma / 2**(self.bits-1)

    y = W_q @ X_q.transpose(-2,-1) #* deq_par
    y = y.transpose(-2,-1)
    return y

In [83]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.key = BitLinear(n_embd, head_size,2)
        self.query = BitLinear(n_embd, head_size,2)
        self.value = BitLinear(n_embd, head_size,2)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * self.head_size **-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [84]:
class MultiHeadAttention(nn.Module):

  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim = -1) #concat over channel dim
    out = self.proj(out)
    return out

In [85]:
class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        BitLinearReLU(n_embd,n_embd,2), # *4 secondo argomento
        nn.ReLU(),
        nn.Linear(n_embd, n_embd), #proj layer #*4 primo argomento
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)

In [86]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [87]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.bitlayer = BitLinear(n_embd,vocab_size,2)


    def forward(self, idx, targets=None):
        B,T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.bitlayer(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [88]:
qmodel = BigramLanguageModel()

In [89]:
optimizer = torch.optim.AdamW(qmodel.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = qmodel(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 7.6065, val loss 7.6757
step 500: train loss 4.1484, val loss 4.2275
step 1000: train loss 4.0566, val loss 4.1290
step 1500: train loss 3.5238, val loss 3.5563
step 2000: train loss 3.2146, val loss 3.2122
step 2500: train loss 3.0099, val loss 2.9947
step 3000: train loss 2.8618, val loss 2.8671
step 3500: train loss 2.8075, val loss 2.7913
step 4000: train loss 2.7687, val loss 2.7556
step 4500: train loss 2.7024, val loss 2.7095


In [70]:
print(decode(qmodel.generate(idx = torch.zeros((1,1),dtype = torch.long), max_new_tokens=500)[0].tolist()))


LNEGhRUKKnNEP:
I cuath wean cealldG :
SHees.
Ilo con, Jthe shoCas oul de loll zoubrsVandast an:
YElCNhardlllcesft
AnlllllS:
xo allllds of th wharGes crNElE$ENIRARNR'PREL:
Loz nand-d fa3DVouBeXPeaXGk
Thxalowh cumc oxst tent
hat co& ;o puX
Q
E'llERSED'Phe mGouateryour Qofis wersw afs
KondY:
Be,X
tqat
HenD ateXlen KIJoA se GoseCe? cJrouzfffeo Illojrd lonl erdt
W whin,
Qily,
PoutOEDSheX 's the wharth che JBouerpeD Bouf$e man$llord
Ture inakealld amenn$ sheramerel?

endG,
ADow Ohishem$
Hhoy whoud, Ma
